In [ ]:
!pip install torch==2.0.1 torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
!pip install portalocker>=2.0.0

In [ ]:
#take the dataset
#mount the dataloader
  #clip the sentences inside the dataloader
  #add eos and sos and prepare the inp and gt

#make the model using gru
#write the train loop and feed the inp and loss with the target
#add extra if for the first batch to display
#make a custom dataset and use the dataloader

In [ ]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader

import torchtext
from torchtext import datasets
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer

import numpy

In [ ]:
train_dataset=datasets.IMDB(split='train')
tokenizer=get_tokenizer("basic_english",language='en')
iter_for_vocab=[]
for label,review in train_dataset:
  iter_for_vocab.append(tokenizer(review))

train_dataset=datasets.IMDB(split='train')


vocab=build_vocab_from_iterator(
    iter_for_vocab,
    specials=["<unk>","<sos>","<eos>","<pad>"],
    special_first=True,
    min_freq=1
)

vocab.set_default_index(vocab["<unk>"])
print("The length of the vocab is ",vocab.__len__())

def text_pipeline(text):
  return vocab.lookup_indices(tokenizer(text))

# print("The number of labels:",len(iter_for_vocab))

The length of the vocab is  68814


In [ ]:
print(text_pipeline("Hello this is custom"))

[3809, 16, 12, 14161]


In [ ]:
def collate_fn(batch):
  input=[]

  for label,review in batch:
    tokens= text_pipeline(review)

    to_put=[]
    if (len(tokens)>10):
      tokens=tokens[:10]

    tokens.insert(0,vocab["<sos>"])
    tokens.append(vocab["<eos>"])

    tokens=torch.tensor(tokens,dtype=torch.long)

    input.append(tokens)

  input=pad_sequence(input,padding_value=vocab["<pad>"],batch_first=True)

  return input


In [ ]:
train_dataloader=DataLoader(train_dataset,batch_size=2,shuffle=True,collate_fn=collate_fn)

In [ ]:
class MyModel(nn.Module):
  def __init__(self,input_size,embed_size,hidden_size):
    super().__init__()
    self.emb=nn.Embedding(input_size,embed_size)
    self.gru=nn.GRU(embed_size,hidden_size,batch_first=True)
    self.lin=nn.Linear(hidden_size,input_size)

  def forward(self,x):
    x=self.emb(x)
    outputs,hidden=self.gru(x)

    outputs=self.lin(outputs)

    return outputs

In [ ]:
model=MyModel(vocab.__len__(),300,512)

In [ ]:
loss_function=nn.CrossEntropyLoss()
optimiser=optim.Adam(model.parameters(),lr=0.01)

In [ ]:
def train_one_epoch():

  model.train()
  train_dataset=datasets.IMDB(split='train')
  train_dataloader=DataLoader(train_dataset,batch_size=2,shuffle=True,collate_fn=collate_fn)

  total_loss=0

  for i,data in enumerate(train_dataloader): #2x12

    input=data[:,:-1]
    gt=data[:,1:]

    if (i==0):
      print("Input pair 1 to model : ",input[0])
      print("Input pair 1 : ",vocab.lookup_tokens(input[0].numpy()))
      print("Output pair 1 to model : ",gt[0])
      print("Target pair 1 : ",vocab.lookup_tokens(gt[0].numpy()))

      print("Input pair 1 to model : ",input[1])
      print("Input pair 1 : ",vocab.lookup_tokens(input[1].numpy()))
      print("Output pair 1 to model : ",gt[1])
      print("Target pair 1 : ",vocab.lookup_tokens(gt[1].numpy()))

    output=model(input)

    # print(output.shape)
    # print(gt.shape)

    loss=loss_function(output.reshape(output.shape[0]*output.shape[1],-1),gt.reshape(-1))

    total_loss+=loss.item()

    print("Loss on batch ",i+1," : ",total_loss/(i+1))

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()


In [ ]:
train_one_epoch()

Input pair 1 to model :  tensor([   1,  241,   16,   29,  280,    4,   14,    9,  134, 1452,   14])
Input pair 1 :  ['<sos>', 'saw', 'this', 'on', 'tv', '.', 'i', "'", 'm', 'glad', 'i']
Output pair 1 to model :  tensor([ 241,   16,   29,  280,    4,   14,    9,  134, 1452,   14,    2])
Target pair 1 :  ['saw', 'this', 'on', 'tv', '.', 'i', "'", 'm', 'glad', 'i', '<eos>']
Input pair 1 to model :  tensor([    1,   503,   113,  1636,     6,  1883,     9, 14580,     9,   212,
            8])
Input pair 1 :  ['<sos>', 'despite', 'its', 'stereotypes', ',', 'virtually', "'", 'no-name', "'", 'cast', 'and']
Output pair 1 to model :  tensor([  503,   113,  1636,     6,  1883,     9, 14580,     9,   212,     8,
            2])
Target pair 1 :  ['despite', 'its', 'stereotypes', ',', 'virtually', "'", 'no-name', "'", 'cast', 'and', '<eos>']
Loss on batch  1  :  8.355917930603027
Loss on batch  2  :  8.570513248443604
Loss on batch  3  :  8.768712997436523
Loss on batch  4  :  8.958849430084229
Loss

KeyboardInterrupt: 

In [ ]:
str1=input("Enter a sample string:")
sample_dataset=[(1,str1)]
sample_dataloader=DataLoader(sample_dataset,batch_size=1,collate_fn=collate_fn)

for data in sample_dataloader:
  print(data)
  print(data.shape)
  print(vocab.lookup_tokens(data[0].numpy()))



Enter a sample string:hello again to another very boring and a very long review and this is fun
tensor([[   1, 3809,  201,   11,  164,   76,  239,    8,    7,   76,  214,    2]])
torch.Size([1, 12])
['<sos>', 'hello', 'again', 'to', 'another', 'very', 'boring', 'and', 'a', 'very', 'long', '<eos>']
